In [1]:
import pandas as pd
import math
from statistics import median
import numpy as np

### (a) Download the MovieLens 100K rating dataset from https://grouplens.org/datasets/movielens/ (the small dataset recommended for education and development). Read the dataset, display the first few rows to understand it, and display the count of ratings (rows) in the dataset to be sure that you download it correctly.

In [2]:
ratings = pd.read_csv("dataset/ratings.csv")
n_rows = ratings.shape[0]

print("Number of rows: " + str(n_rows))
# Show the first 10 rows of the dataframe
print(ratings.head(10).to_string(index=False))

Number of rows: 100836
 userId  movieId  rating  timestamp
      1        1     4.0  964982703
      1        3     4.0  964981247
      1        6     4.0  964982224
      1       47     5.0  964983815
      1       50     5.0  964982931
      1       70     3.0  964982400
      1      101     5.0  964980868
      1      110     4.0  964982176
      1      151     5.0  964984041
      1      157     5.0  964984100


In [3]:
movies = pd.read_csv("dataset/movies.csv")
n_rows = movies.shape[0]

print("Number of rows: " + str(n_rows))
# Show the first 10 rows of the dataframe
print(movies.head(10).to_string(index=False))

Number of rows: 9742
 movieId                              title                                      genres
       1                   Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
       2                     Jumanji (1995)                  Adventure|Children|Fantasy
       3            Grumpier Old Men (1995)                              Comedy|Romance
       4           Waiting to Exhale (1995)                        Comedy|Drama|Romance
       5 Father of the Bride Part II (1995)                                      Comedy
       6                        Heat (1995)                       Action|Crime|Thriller
       7                     Sabrina (1995)                              Comedy|Romance
       8                Tom and Huck (1995)                          Adventure|Children
       9                Sudden Death (1995)                                      Action
      10                   GoldenEye (1995)                   Action|Adventure|Thriller


In [4]:
def create_user_movie_rating_matrix():
    user_movie_ratings_matrix = pd.pivot_table(ratings, values='rating', index='userId', columns='movieId')
    all_movie_ids = movies['movieId'].unique()
    user_movie_ratings_matrix = user_movie_ratings_matrix.reindex(columns=all_movie_ids)
    return user_movie_ratings_matrix

In [6]:
user_movie_ratings_matrix = create_user_movie_rating_matrix()
user_movie_ratings_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### (b) Implement the user-based collaborative filtering approach, using the Pearson correlation function for computing similarities between users, and

$$sim(a, b) = \frac{\sum_{p \in I} (r_{a,p} - \bar{r_a})(r_{b,p} - \bar{r_b})}{\sqrt{\sum_{p \in I}(r_{a,p} - \bar{r_a})^2}\sqrt{\sum_{p \in I}(r_{b,p} - \bar{r_b})^2}}$$

In [7]:
num_users = ratings["userId"].nunique()
print(num_users)

610


In [8]:
def pearsonCorrelation(user1, user2):
    num, den1, den2 = 0.0, 0.0, 0.0
    ratings_user1 = ratings[ratings['userId'] == user1]
    rmean_user1 = ratings_user1['rating'].mean()
    ratings_user2 = ratings[ratings['userId'] == user2]
    rmean_user2 = ratings_user2['rating'].mean()

    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1.intersection(items_user2)

    for p in common_items:
        r_1p = (ratings[(ratings['userId'] == user1) & (ratings['movieId'] == p)]['rating'].values)[0]
        r_2p = (ratings[(ratings['userId'] == user2) & (ratings['movieId'] == p)]['rating'].values)[0]
        num += (r_1p - rmean_user1)*(r_2p - rmean_user2)
        den1 += pow((r_1p - rmean_user1), 2)
        den2 += pow((r_2p - rmean_user2), 2)
    den = ((math.sqrt(den1))*(math.sqrt(den2)))
    if den == 0.0:
        return 0.0
    sim = num/den
    return sim

In [9]:
corr = pearsonCorrelation(1,21)

print(corr)

0.08648178161396775


In [10]:
def compute_user_similarities(user):
    user_list = list(range(1, 611, 1))
    user_list.remove(user)
    user_similarities = {}
    for other_user in user_list:
        if other_user != user:
            similarity = pearsonCorrelation(user, other_user)
            user_similarities[other_user] = similarity
    return user_similarities

In [184]:
user_similarities = compute_user_similarities(1)
user_similarities

{2: 0.9999999999999998,
 3: 0.011225168459991653,
 4: 0.210423287816751,
 5: 0.1947213927869427,
 6: -0.3020187823310266,
 7: -0.04664156383054844,
 8: 0.4265612329426459,
 9: 0.3547385682753224,
 10: -0.15121755406386428,
 11: -0.4499572307079906,
 12: 1.0,
 13: 0.9478788458420678,
 14: 0.2253535048415333,
 15: 0.3120484635421993,
 16: 0.04963408429210114,
 17: 0.19844179978497972,
 18: 0.23106109729152322,
 19: 0.22574845026891835,
 20: 0.5256863271011928,
 21: 0.08648178161396775,
 22: -0.1302706228212957,
 23: -0.2606849822350735,
 24: 0.05176459800704182,
 25: 0.018122806444241503,
 26: -0.14770835432617932,
 27: 0.18060517781572213,
 28: 0.009072639414787047,
 29: -0.0632480567160467,
 30: 0.05197519878093252,
 31: 0.052404952231923506,
 32: 0.270074390572318,
 33: 0.10531918357689721,
 34: 0.10009627479532733,
 35: 0.31319277272972756,
 36: 0.44197925638731156,
 37: -0.41904339197822715,
 38: 0.058071462757481596,
 39: -0.2254567309877417,
 40: -0.35522450414630213,
 41: -0.1427

### (c) the prediction function presented in class for predicting movies scores.

$$pred(a, p) = \bar{r_a} + \frac{\sum_{b \in N} sim(a, b) * (r_{b,p} - \bar{r_b})} {\sum_{b \in N}sim(a, b)}$$

In [11]:
def generatePrediction(user1, item, user_movie_ratings_matrix, user_similarity_dict):
    # Aggiungi controllo che il film non è stato valutato
    if not np.isnan(user_movie_ratings_matrix.at[user1, item]):
        return user_movie_ratings_matrix.at[user1, item]
    
    # Calcola la previsione utilizzando la matrice delle valutazioni e il dizionario delle similarità
    num, den = 0.0, 0.0
    rmean_user1 = user_movie_ratings_matrix.loc[user1].mean()
    
    users_who_rated_item = user_movie_ratings_matrix[item].dropna().index.unique()
    for u in users_who_rated_item:
        rmean_u = user_movie_ratings_matrix.loc[u].mean()
        r_up = user_movie_ratings_matrix.at[u, item]
        similarity = user_similarity_dict[u]
        num += similarity * (r_up - rmean_u)
        den += abs(similarity)
    if den == 0.0:
        pred = rmean_user1
    else:
        pred = rmean_user1 + (num / den)
    return pred

In [20]:
print(generatePrediction(1,5105,create_user_movie_rating_matrix(),compute_user_similarities(1)))

7.790948275862069


### (d) Select a user from the dataset, and for this user, show the 10 most similar users and the 10 most relevant movies that the recommender suggests.

In [187]:
def kMostSimilarUsers(user, k):
    users_similarities = []
    for u in range(1,num_users):
        if u != user:
            sim = pearsonCorrelation(user, u)
            users_similarities.append((u,sim))
    sorted_users_similarities = sorted(users_similarities, reverse=True, key=lambda x: x[1])
    top_k_users = [pair[0] for pair in sorted_users_similarities[:k]]
    top_k_sim = [pair[1] for pair in sorted_users_similarities[:k]]
    return top_k_users, top_k_sim

In [188]:
print(kMostSimilarUsers(3, 10))

([14, 34, 55, 65, 73, 75, 80, 82, 98, 121], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


In [14]:
def get_unrated_movie_ids(userId):
    # Ottieni tutti i movieId unici presenti nel DataFrame ratings
    all_movie_ids = movies['movieId'].tolist()
    
    # Ottieni i movieId valutati dal userId specificato
    rated_movie_ids = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    # Trova i movieId non valutati dal userId
    unrated_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids]
    
    return unrated_movie_ids

In [15]:
unrated_movies = get_unrated_movie_ids(1)
print(unrated_movies)

[2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 52, 53, 54, 55, 57, 58, 60, 61, 62, 63, 64, 65, 66, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 99, 100, 102, 103, 104, 105, 106, 107, 108, 111, 112, 113, 116, 117, 118, 119, 121, 122, 123, 125, 126, 128, 129, 132, 135, 137, 140, 141, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 217, 218, 219, 220, 222, 224, 225, 227, 228, 229, 230, 232, 233, 234, 236, 237, 238, 239, 240, 241, 242, 243, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 263, 265, 266, 267, 269, 270, 27

In [18]:
def kMostRelevantMovies(user, k):
    user_movie_ratings_matrix = create_user_movie_rating_matrix()
    unrated_movies = get_unrated_movie_ids(user)
    predictions = []
    user_similarities_dict = compute_user_similarities(user)
    for m in unrated_movies:
        print("m: "+str(m))
        pred = generatePrediction(user, m, user_movie_ratings_matrix,user_similarities_dict)
        predictions.append((m, pred)) 
    sorted_predictions = sorted(predictions, reverse=True, key=lambda x: x[1])
    top_k_predictions = sorted_predictions[:k]
    return top_k_predictions

In [19]:
print(kMostRelevantMovies(1,10))

m: 2
m: 4
m: 5
m: 7
m: 8
m: 9
m: 10
m: 11
m: 12
m: 13
m: 14
m: 15
m: 16
m: 17
m: 18
m: 19
m: 20
m: 21
m: 22
m: 23
m: 24
m: 25
m: 26
m: 27
m: 28
m: 29
m: 30
m: 31
m: 32
m: 34
m: 36
m: 38
m: 39
m: 40
m: 41
m: 42
m: 43
m: 44
m: 45
m: 46
m: 48
m: 49
m: 52
m: 53
m: 54
m: 55
m: 57
m: 58
m: 60
m: 61
m: 62
m: 63
m: 64
m: 65
m: 66
m: 68
m: 69
m: 71
m: 72
m: 73
m: 74
m: 75
m: 76
m: 77
m: 78
m: 79
m: 80
m: 81
m: 82
m: 83
m: 85
m: 86
m: 87
m: 88
m: 89
m: 92
m: 93
m: 94
m: 95
m: 96
m: 97
m: 99
m: 100
m: 102
m: 103
m: 104
m: 105
m: 106
m: 107
m: 108
m: 111
m: 112
m: 113
m: 116
m: 117
m: 118
m: 119
m: 121
m: 122
m: 123
m: 125
m: 126
m: 128
m: 129
m: 132
m: 135
m: 137
m: 140
m: 141
m: 144
m: 145
m: 146
m: 147
m: 148
m: 149
m: 150
m: 152
m: 153
m: 154
m: 155
m: 156
m: 158
m: 159
m: 160
m: 161
m: 162
m: 164
m: 165
m: 166
m: 168
m: 169
m: 170
m: 171
m: 172
m: 173
m: 174
m: 175
m: 176
m: 177
m: 178
m: 179
m: 180
m: 181
m: 183
m: 184
m: 185
m: 186
m: 187
m: 188
m: 189
m: 190
m: 191
m: 193
m: 194
m: 195
m: 

### (e) Design and implement a new similarity function for computing similarities between users. Explain why this similarity function is useful for the collaborative filtering approach. Hint: Exploiting ideas from related works are highly encouraged.

#### Constrained Pearson Correlation Coefficient (CPCC)
$$sim(a, b)^{CPCC} = \frac{\sum_{p \in I} (r_{a,p} - r_{med})(r_{b,p} - r_{med})}{\sqrt{\sum_{p \in I}(r_{a,p} - r_{med})^2}\sqrt{\sum_{p \in I}(r_{b,p} - r_{med})^2}}$$

In [193]:
def constrainedPearsonCorrelation(user1, user2):
    num, den1, den2 = 0.0, 0.0, 0.0
    possible_ratings = ratings['rating'].unique().tolist()
    possible_ratings.sort()
    print(possible_ratings)
    median_value = median(possible_ratings)
    print(median_value)

    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1.intersection(items_user2)

    for p in common_items:
        r_1p = (ratings[(ratings['userId'] == user1) & (ratings['movieId'] == p)]['rating'].values)[0]
        r_2p = (ratings[(ratings['userId'] == user2) & (ratings['movieId'] == p)]['rating'].values)[0]
        num += (r_1p - median_value)*(r_2p - median_value)
        den1 += pow((r_1p - median_value), 2)
        den2 += pow((r_2p - median_value), 2)
    den = ((math.sqrt(den1))*(math.sqrt(den2)))
    if den == 0.0:
        return 0.0
    sim = num/den
    return sim

In [194]:
corr = constrainedPearsonCorrelation(1,21)

print(corr)

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
2.75
0.6593415600314004


#### Weighted Pearson Correlation Coefficient (WPCC)
$$ sim(a,b)^{WPCC}=\begin{cases} sim(a,b)^{PCC}*\frac{|I|}{H}, & |I|\le{H} \\ sim(a,b)^{PCC}, & otherwise
\end{cases} $$
##### where I represents the set of common rating items by user u and v and H is an experimental value and it is set 50 

In [195]:
def weightedPearsonCorrelation(user1, user2):
    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1.intersection(items_user2)
    
    similarity = pearsonCorrelation(user1, user2)
    I = len(common_items)
    H = 50
    if I <= H:
        sim = similarity * (I/H)
    else:
        sim = similarity
    return sim

In [196]:
corr = weightedPearsonCorrelation(1,21)

print(corr)

0.08129287471712968


#### Sigmoid Function based Pearson correlation coefficient (SPCC) 
$$sim(a, b)^{SPCC} = sim(a, b)^{PCC} * \frac{1}{1 + \exp{-(\frac{|I|}{2})}}$$

In [197]:
def sigmoidFunctionBasedPearsonCorrelation(user1, user2):
    similarity = pearsonCorrelation(user1,user2)
    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1.intersection(items_user2)
    I = len(common_items)
    sim = similarity * (1/(1+math.exp(-(I/2))))
    return sim

In [198]:
corr = sigmoidFunctionBasedPearsonCorrelation(1,21)

print(corr)

0.08648178160858498


####  Adjusted Cosine Measure
$$sim(a, b)^{ACOS} = \frac{\sum_{p \in P} (r_{a,p} - \bar{r_a})(r_{b,p} - \bar{r_b})}{\sqrt{\sum_{p \in P}(r_{a,p} - \bar{r_a})^2}\sqrt{\sum_{p \in P}(r_{b,p} - \bar{r_b})^2}}$$
##### where $P$ is the set of all items. If user $u$ has not rated the item $p∈P$, the rating $r_{u,p}$ is zero.

In [199]:
def adjustedCosineMeasure(user1, user2):
    num, den1, den2 = 0.0, 0.0, 0.0
    ratings_user1 = ratings[ratings['userId'] == user1]
    rmean_user1 = ratings_user1['rating'].mean()
    ratings_user2 = ratings[ratings['userId'] == user2]
    rmean_user2 = ratings_user2['rating'].mean()

    all_items = movies['movieId'].tolist()
    
    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])

    for p in all_items:
        if p not in items_user1:
            r_1p = 0.0
        else:
            r_1p = (ratings[(ratings['userId'] == user1) & (ratings['movieId'] == p)]['rating'].values)[0]
        if p not in items_user2:
            r_2p = 0.0
        else:
            r_2p = (ratings[(ratings['userId'] == user2) & (ratings['movieId'] == p)]['rating'].values)[0]
        num += (r_1p - rmean_user1)*(r_2p - rmean_user2)
        den1 += pow((r_1p - rmean_user1), 2)
        den2 += pow((r_2p - rmean_user2), 2)
    den = ((math.sqrt(den1))*(math.sqrt(den2)))
    if den == 0.0:
        return 0.0
    sim = num/den
    return sim

In [200]:
corr = adjustedCosineMeasure(1,21)

print(corr)

0.9671568115412114


#### Jaccard
$$sim(a, b)^{Jaccard} = \frac{|I_a| \cap {|I_b|}}{|I_a| \cup {|I_b|}}$$

In [201]:
def jaccard(user1, user2):
    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])

    intersection = len(items_user1.intersection(items_user2))
    union = len(items_user1.union(items_user2))
    return intersection / union

In [202]:
corr = jaccard(1,21)

print(corr)

0.07484076433121019


#### Mean Squared Difference (MSD)
$$sim(a, b)^{MSD} = 1 - \frac{\sum_{p \in I} (r_{a,p} - r_{b,p})^2}{|I|}$$

In [203]:
def meanSquaredDifference(user1, user2):
    num = 0.0
    # set of movieIds evaluated by user1
    items_user1 = set(ratings[ratings['userId'] == user1]['movieId'])
    # set of movieIds evaluated by user2
    items_user2 = set(ratings[ratings['userId'] == user2]['movieId'])
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1.intersection(items_user2)
    I = len(common_items)

    for p in common_items:    
        r_1p = (ratings[(ratings['userId'] == user1) & (ratings['movieId'] == p)]['rating'].values)[0]
        r_2p = (ratings[(ratings['userId'] == user2) & (ratings['movieId'] == p)]['rating'].values)[0]
        num += pow((r_1p - r_2p), 2)
    
    if I != 0:
        sim = 1 - (num / I)
    else:
        sim = 0.0
    return sim
    

In [204]:
corr = meanSquaredDifference(1,21)

print(corr)

-0.7606382978723405


####  Jaccard and MSD can be combined to form a new metric. 
$$sim(a, b)^{JMSD} = sim(a, b)^{Jaccard}*sim(a, b)^{MSD}

In [205]:
def JaccardAndMSD(user1, user2):
    return jaccard(user1,user2)*meanSquaredDifference(user1,user2)

In [206]:
corr = JaccardAndMSD(1,21)

print(corr)

-0.05692675159235669
